In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from synthetic import simulate_lorenz_96
from models.clstm import cLSTM, train_model_ista

import pandas as pd

In [3]:
device = torch.device('cuda')

In [12]:
PATH = 'dataset/train/v_-6Os86HzwCs_c009/'

gt_path = PATH + 'gt/gt.txt'

df = pd.read_csv(gt_path, sep=',', header=None)

df.head()

,0,1,2,3,4,5,6,7,8
0,1,0,102,271,43,116,1,1,1
1,1,1,211,403,110,138,1,1,1
2,1,2,297,449,88,129,1,1,1
3,1,3,110,248,34,104,1,1,1
4,1,4,383,257,55,91,1,1,1


In [15]:
import numpy as np
from itertools import combinations

df.sort_values(by=0, inplace=True)

unique_frames = df[0].unique()

def get_players_in_scope(df, start_frame, end_frame):
    frames_in_scope = df[df[0].between(start_frame, end_frame)]
    player_counts = frames_in_scope.groupby(1)[0].count()
    qualified_players = player_counts[player_counts == (end_frame - start_frame + 1)]
    return qualified_players.index.tolist()

longest_scope = (0, 0)
players_in_longest_scope = []

for start_frame, end_frame in combinations(unique_frames, 2):
    if end_frame - start_frame > longest_scope[1] - longest_scope[0]:
        players = get_players_in_scope(df, start_frame, end_frame)
        if len(players) >= 3:
            longest_scope = (start_frame, end_frame)
            players_in_longest_scope = players

all_possible_players = players_in_longest_scope
selected_players = np.random.choice(players_in_longest_scope, 3, replace=False) if len(players_in_longest_scope) > 3 else players_in_longest_scope

print("Longest continuous frame scope:", longest_scope)
print("All possible player IDs in the scope:", all_possible_players)
print("Selected player IDs:", selected_players)

Longest continuous frame scope: (1, 500)
All possible player IDs in the scope: [0, 1, 2, 3, 4, 6, 7, 8, 9]
Selected player IDs: [2 4 0]


In [18]:
df['central_point_x'] = df[2] + df[4] / 2
df['central_point_y'] = df[3] + df[5] / 2

In [19]:
df.head()

,0,1,2,3,4,5,6,7,8,central_point_x,central_point_y
0,1,0,102,271,43,116,1,1,1,123.5,329.0
1,1,1,211,403,110,138,1,1,1,266.0,472.0
2,1,2,297,449,88,129,1,1,1,341.0,513.5
3,1,3,110,248,34,104,1,1,1,127.0,300.0
4,1,4,383,257,55,91,1,1,1,410.5,302.5


In [26]:
df_chosen_players = df[df[1].isin(selected_players)]
df_chosen_players.head()

,0,1,2,3,4,5,6,7,8,central_point_x,central_point_y
0,1,0,102,271,43,116,1,1,1,123.5,329.0
2,1,2,297,449,88,129,1,1,1,341.0,513.5
4,1,4,383,257,55,91,1,1,1,410.5,302.5
10,2,0,110,271,43,116,1,1,1,131.5,329.0
14,2,4,391,256,53,92,1,1,1,417.5,302.0


In [27]:
pivot_x = df_chosen_players.pivot(index=0, columns=1, values='central_point_x').fillna(0)
pivot_y = df_chosen_players.pivot(index=0, columns=1, values='central_point_y').fillna(0)

In [28]:
tensor_x = torch.tensor(pivot_x.values[None, :, :], dtype=torch.float32)
tensor_y = torch.tensor(pivot_y.values[None, :, :], dtype=torch.float32)

print(tensor_x.shape)  
print(tensor_y.shape)

torch.Size([1, 500, 3])
torch.Size([1, 500, 3])


In [30]:
X = tensor_x.to(device)

clstm = cLSTM(X.shape[-1], hidden=100).cuda(device=device)

train_loss_list = train_model_ista(
    clstm, X, context=10, lam=10.0, lam_ridge=1e-2, lr=1e-3, max_iter=20000,
    check_every=50)

----------Iter = 50----------
Loss = 28747.943359
Variable usage = 100.00%
----------Iter = 100----------
Loss = 24803.582031
Variable usage = 100.00%
----------Iter = 150----------
Loss = 24517.988281
Variable usage = 100.00%
----------Iter = 200----------
Loss = 24596.417969
Variable usage = 100.00%
----------Iter = 250----------
Loss = 24729.976562
Variable usage = 100.00%
----------Iter = 300----------
Loss = 24695.287109
Variable usage = 100.00%
----------Iter = 350----------
Loss = 24742.425781
Variable usage = 100.00%
----------Iter = 400----------
Loss = 24912.810547
Variable usage = 100.00%
Stopping early
